In [1]:
#Código para el TFM de Víctor Gabaldón Moreno (vicgabal@ucm.es) sobre Análisis de la contaminación lumínica en Madrid usando imagenes desde el espacio
#Encuentra las fuentes puntuales o hotspots de la imagen de referencia en el canal verde y luego realiza fotometría de apertura usando esas detecciones de referencia para los demás canales.
#Genera 1 archivo por canal donde registra el flujo en las aperturas.

In [ ]:
import pandas as pd
from astropy.io import ascii
import photutils
import astropy
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.figure as f
import astropy.io.fits as fits
from math import pi
import pandas
import scipy
from scipy import stats
from scipy.optimize import curve_fit
from numpy import exp,arange
from pylab import meshgrid,cm,imshow,contour,clabel,colorbar,axis,title,show
import lmfit as lm
from photutils import detect_threshold
from astropy.convolution import Gaussian2DKernel
from astropy.stats import gaussian_fwhm_to_sigma
from photutils import detect_sources
from astropy.visualization import SqrtStretch
from astropy.visualization.mpl_normalize import ImageNormalize
import os,sys,glob
import matplotlib as mpl
from matplotlib.pyplot import figure, show, rc, grid, savefig
from matplotlib.ticker import MultipleLocator
import numpy as np
from pylab import *
from matplotlib.ticker import MultipleLocator, FormatStrFormatter
from astropy.stats import sigma_clipped_stats
from astropy.stats import SigmaClip
from photutils import Background2D, MedianBackground
from astropy.stats import sigma_clipped_stats
from photutils import datasets
from photutils import DAOStarFinder
from astropy.visualization import SqrtStretch
from astropy.visualization.mpl_normalize import ImageNormalize
from photutils import CircularAperture
from astropy.wcs import WCS
from photutils import CircularAperture
from photutils import aperture_photometry

dpi=250
plt.rcParams['figure.dpi']= dpi

#%matplotlib qt

In [3]:
#ALGORITMO DE DETECCION DE FUENTES PUNTUALES PARA LA IMAGEN DE REFERENCIA:

directory_load = '.\Ficheros_fits_test\\'
referencia = 'Corr_iss062e102209G2F.cal2N.Ao2_rect.fits'
img_referencia = fits.open(directory_load+referencia)
data_referencia = img_referencia[0].data

#Fondo de la imagen de referencia:

sigma_clip_referencia = SigmaClip(sigma=4)#(2.0 antes) number of standard deviations (per pixel) above the background
bkg_estimator_referencia = MedianBackground()
mask = (data_referencia == 0)
bkg_referencia = Background2D(data_referencia, (15, 15), filter_size=(3, 3), mask=mask)
fondo_referencia = bkg_referencia.background * ~mask
imagen_sin_fondo_referencia=data_referencia-fondo_referencia

#busqueda de fuentes

mean, median, std = sigma_clipped_stats(imagen_sin_fondo_referencia, sigma=5)    
daofind_referencia = DAOStarFinder(fwhm=6, threshold=70.*std)  #FWHM anchura a media altura. Más grande= fuente de luz cae más suave
sources_referencia = daofind_referencia(imagen_sin_fondo_referencia - median) 

for col in sources_referencia.colnames:  
    sources_referencia[col].info.format = '%.8g'  # for consistent table output

positions_referencia = np.transpose((sources_referencia['xcentroid'], sources_referencia['ycentroid']))
apertures_referencia = CircularAperture(positions_referencia, r=4.)

#GUARDADO DEL ARCHIVO .dat con la tabla de fuentes:

ascii.write(sources_referencia, str(referencia)+'.dat', overwrite=True)
    
#Añadido de coordenadas de lon y lat:

tabla_referencia=pd.read_table(str(referencia)+'.dat', sep=' ')

#LEER LAS COORDENADAS Y CONVERTIRLAS A GRADOS CENTESIMALES

xcentroid=tabla_referencia['xcentroid'] #LEER las x
ycentroid=tabla_referencia['ycentroid'] #Leer las y

wcosa_referencia = WCS(directory_load+referencia)
lon, lat = wcosa_referencia.all_pix2world(xcentroid, ycentroid, 0)
lon=lon-360

tabla_referencia['lon'] = lon  #AÑADIR LA COLUMNA x CONVERTIDA A LA TABLA
tabla_referencia['lat'] = lat #AÑADIR LA COLUMNA y CONVERTIDA A LA TABLA

tabla_referencia.to_csv('.\Fuentes_puntuales_referencia_verde\\'+'referencia'+str(referencia)+'.csv',sep=";",index=False)
print('>>>>>>>>>>>>>>>>>>>>>>>>>>>>Creado el fichero '+'referencia'+str(referencia)+'.csv')
    
borrar=os.listdir('.')

for item in borrar:
    if item.endswith(".dat"):
        os.remove(item)
        
print(tabla_referencia)

>>>>>>>>>>>>>>>>>>>>>>>>>>>>Creado el fichero referenciaCorr_iss062e102209G2F.cal2N.Ao2_rect.fits.csv
      id   xcentroid   ycentroid  sharpness  roundness1  roundness2  npix  \
0      1  1099.69420   636.91767   0.361568   -0.733243   -0.217923    49   
1      2   978.00866  1078.38020   0.639675   -0.144279    0.068182    49   
2      3   828.82897  1216.27160   0.572201   -0.689529   -0.144790    49   
3      4   826.14590  1219.43370   0.499533    0.224735    0.471627    49   
4      5   844.18899  1230.29760   0.488798    0.043228    0.521383    49   
..   ...         ...         ...        ...         ...         ...   ...   
344  345  1030.93380  2549.68010   0.716955    0.607132    0.381387    49   
345  346  1011.91150  2561.19230   0.505998   -0.574696   -0.032520    49   
346  347  1009.05390  2563.65540   0.821499   -0.649429    0.080656    49   
347  348   980.87929  2565.60480   0.575917   -0.424672   -0.361886    49   
348  349   987.49707  2707.27410   0.448696    0.24

In [4]:
#FOTOMETRÍA DE APERTURA PARA LOS 4 CANALES DE Corr_iss062e102209

directory_load = '.\Ficheros_fits_test\\'
referencia = 'Corr_iss062e102209G2F.cal2N.Ao2_rect.fits'
tabla_referencia=pd.read_table('.\Fuentes_puntuales_referencia_verde\\'+'referencia'+str(referencia)+'.csv', sep=';')

v=np.arange(0,len(tabla_referencia['xcentroid']),1)
positions = []
for n in v:
    position = (tabla_referencia.loc[n,'xcentroid'], tabla_referencia.loc[n,'ycentroid'])
    positions.append(position)
apertures = CircularAperture(positions, r=4)

files = [i for i in os.listdir(directory_load) if i.startswith("Corr_iss062e102209")]

#TRATAMIENTO DEL FONDO

for index, file in enumerate(files):
    img = fits.open(directory_load+file)
    data = img[0].data
    sigma_clip = SigmaClip(sigma=4)#(2.0 antes) number of standard deviations (per pixel) above the background
    bkg_estimator = MedianBackground()
    mask = (data == 0)
    bkg = Background2D(data, (15, 15), filter_size=(3, 3), mask=mask)
    fondo = bkg.background * ~mask
    imagen_sin_fondo=data-fondo

#FOTOMETRÍA DE APERTURA con r=4
    phot_table = aperture_photometry(imagen_sin_fondo, apertures)
    phot_table['aperture_sum'].info.format = '%.8g'  # for consistent table output
    ascii.write(phot_table, '.\Fuentes_puntuales_referencia_verde\\'+str(file)+'.dat', overwrite=True)

#Añadido de coordenadas de lon y lat:

    phot_table=pd.read_table('.\Fuentes_puntuales_referencia_verde\\'+str(file)+'.dat', sep=' ')

#LEER LAS COORDENADAS Y CONVERTIRLAS A GRADOS CENTESIMALES

    xcenter=phot_table['xcenter'] #LEER las x
    ycenter=phot_table['ycenter'] #Leer las y
    wcosa = WCS(directory_load+file)
    lon, lat = wcosa.all_pix2world(xcenter, ycenter, 0)
    lon=lon-360

    phot_table['lon'] = lon  #AÑADIR LA COLUMNA x CONVERTIDA A LA TABLA
    phot_table['lat'] = lat #AÑADIR LA COLUMNA y CONVERTIDA A LA TABLA

    phot_table.to_csv('.\Fuentes_puntuales_referencia_verde\\'+str(file)+'.csv',sep=";",index=False)

    print('>>>>>>>>>>>>>>>>>>>>>>>>>>>>Creado el fichero '+str(file)+'.csv')

>>>>>>>>>>>>>>>>>>>>>>>>>>>>Creado el fichero Corr_iss062e102209B4F.cal2N.Ao2_rect.fits.csv
>>>>>>>>>>>>>>>>>>>>>>>>>>>>Creado el fichero Corr_iss062e102209G2F.cal2N.Ao2_rect.fits.csv
>>>>>>>>>>>>>>>>>>>>>>>>>>>>Creado el fichero Corr_iss062e102209G3F.cal2N.Ao2_rect.fits.csv
>>>>>>>>>>>>>>>>>>>>>>>>>>>>Creado el fichero Corr_iss062e102209R1F.cal2N.Ao2_rect.fits.csv
